In [ ]:
import numpy as np
%matplotlib notebook
from func import *

In [ ]:
SPEED = 0
THETA = 1
T_DROP = 2
T_DETONATE = 3

$$
\arg\max t = \arg\max(t-\alpha) = \arg\min(\alpha - t)
$$

In [ ]:
def loss_func(x):
    speed, theta, t_drop, t_detonate = x
    set_fy_mission(0, speed, theta)
    interval = get_time_interval(0, 0, t_drop, t_detonate)
    period = get_lebesgue(interval)
    loss = -period
    return loss

In [ ]:
initial_params = [120, 3.141592653589793, 1.5, 3.6]
bounds = [(fy_speed_min, fy_speed_max), (0, 2 * np.pi), (0, missile_life[0]), (0, missile_life[0])]

In [ ]:
from scipy.optimize import minimize
methods = ['L-BFGS-B', 'SLSQP', 'TNC']  # 这些方法都支持边界约束

best_result = None
best_loss = float('inf')

for method in methods:
    try:
        result = minimize(
            loss_func,
            initial_params,
            method=method,
            bounds=bounds,
            options={'maxiter': 10000, 'disp': True}
        )

        if result.success and result.fun < best_loss:
            best_result = result
            best_loss = result.fun

    except Exception as e:
        print(f"Method {method} failed: {e}")

if best_result is not None:
    print("优化结果:")
    print(f"最优参数: {best_result.x}")
    print(f"最小损失值: {best_result.fun}")
    print(f"优化是否成功: {best_result.success}")
    print(f"迭代次数: {best_result.nit}")
    print(f"函数调用次数: {best_result.nfev}")
else:
    print("所有优化方法都失败了")

# 如果你想要使用梯度下降（虽然对于有边界的问题，上述方法更好）
# 你也可以尝试使用带投影的梯度下降
from scipy.optimize import minimize

# 使用投影函数确保参数在边界内
def project_to_bounds(x):
    x_projected = x.copy()
    x_projected[0] = np.clip(x[0], bounds[0][0], bounds[0][1])
    x_projected[1] = np.clip(x[1], bounds[1][0], bounds[1][1])
    x_projected[2] = max(x[2], bounds[2][0])
    x_projected[3] = max(x[3], bounds[3][0])
    return x_projected

# 使用无约束优化方法+投影
result_gd = minimize(
    lambda x: loss_func(project_to_bounds(x)),
    initial_params,
    method='BFGS',  # 或者 'CG'
    options={'maxiter': 10000, 'disp': True}
)

# 将结果投影回边界
optimized_params = project_to_bounds(result_gd.x)
print(f"梯度下降优化结果: {optimized_params}")
print(f"最终损失值: {loss_func(optimized_params)}")

In [ ]:
with content:
    loss_func(optimized_params)
draw_content()